In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
from datetime import datetime, timedelta
%matplotlib inline

In [2]:
#source_master = Path("../Resources/choice_df.csv")

source_msft = Path("Resources/master_msft.csv")

#master_df = pd.read_csv(source_master)
msft_df = pd.read_csv(source_msft)


In [3]:
msft_df['ExpirationDate'] =  pd.to_datetime(msft_df['ExpirationDate'], infer_datetime_format=True)
msft_df['DataDate'] =  pd.to_datetime(msft_df['DataDate'], infer_datetime_format=True)
msft_df['time delta'] = msft_df['ExpirationDate'] - msft_df['DataDate']
msft_df['% out'] = msft_df['StrikePrice'] / msft_df['UnderlyingPrice'] - 1

msft_df.tail()

,Unnamed: 0,OptionKey,Symbol,ExpirationDate,AskPrice,AskSize,BidPrice,BidSize,LastPrice,PutCall,StrikePrice,Volume,OpenInterest,UnderlyingPrice,DataDate,time delta,% out
76631,526763,MSFT2021-03-19C135.002019-06-19,MSFT,2021-03-19,19.70,222,17.10,71,17.99,call,135.0,10,156,135.69,2019-06-19,639 days,-0.005085
76632,526764,MSFT2021-03-19C140.002019-06-19,MSFT,2021-03-19,17.35,220,14.45,126,14.42,call,140.0,1,55,135.69,2019-06-19,639 days,0.031764
76633,526765,MSFT2021-03-19C145.002019-06-19,MSFT,2021-03-19,15.10,220,12.60,102,12.40,call,145.0,1,128,135.69,2019-06-19,639 days,0.068612
76634,526767,MSFT2021-03-19C155.002019-06-19,MSFT,2021-03-19,11.35,204,8.80,121,9.10,call,155.0,1,161,135.69,2019-06-19,639 days,0.142310
76635,526811,MSFT2021-03-19P95.002019-06-19,MSFT,2021-03-19,4.00,64,2.64,246,3.85,put,95.0,1,46,135.69,2019-06-19,639 days,-0.299875


In [4]:
df = pd.DataFrame(msft_df)
df = df.set_index(pd.to_datetime(df.DataDate, infer_datetime_format=True))
df['OptionKey'] = df['OptionKey'].map(lambda x:str(x)[:-10])
df = df.drop(columns = ['Unnamed: 0', 'AskSize', 'BidSize'])

df.head()

,OptionKey,Symbol,ExpirationDate,AskPrice,BidPrice,LastPrice,PutCall,StrikePrice,Volume,OpenInterest,UnderlyingPrice,DataDate,time delta,% out
DataDate,,,,,,,,,,,,,,
2019-06-20,MSFT2019-08-02C124.00,MSFT,2019-08-02,14.60,13.85,13.79,call,124.0,1,0,136.95,2019-06-20,43 days,-0.094560
2019-06-20,MSFT2019-08-02C125.00,MSFT,2019-08-02,13.35,12.95,12.32,call,125.0,1,6,136.95,2019-06-20,43 days,-0.087258
2019-06-20,MSFT2019-08-02C127.00,MSFT,2019-08-02,11.60,11.15,11.37,call,127.0,25,15,136.95,2019-06-20,43 days,-0.072654
2019-06-20,MSFT2019-08-02C130.00,MSFT,2019-08-02,9.05,8.85,9.15,call,130.0,4,62,136.95,2019-06-20,43 days,-0.050748
2019-06-20,MSFT2019-08-02C131.00,MSFT,2019-08-02,8.40,8.00,7.50,call,131.0,40,103,136.95,2019-06-20,43 days,-0.043447


In [5]:
#track PnL code- filter for only puts, slice dataframe length and sort
df = df[(df['ExpirationDate'] <= '2019-12-30') & (df.PutCall == 'put')]
df = df.sort_values(['OptionKey','time delta'])
                                                   
df.head()

,OptionKey,Symbol,ExpirationDate,AskPrice,BidPrice,LastPrice,PutCall,StrikePrice,Volume,OpenInterest,UnderlyingPrice,DataDate,time delta,% out
DataDate,,,,,,,,,,,,,,
2019-06-06,MSFT2019-06-07P105.00,MSFT,2019-06-07,0.02,0.0,0.01,put,105.0,15,0,127.82,2019-06-06,1 days,-0.178532
2019-06-07,MSFT2019-06-07P108.00,MSFT,2019-06-07,0.01,0.0,0.01,put,108.0,10,533,131.40,2019-06-07,0 days,-0.178082
2019-06-06,MSFT2019-06-07P113.00,MSFT,2019-06-07,0.02,0.0,0.00,put,113.0,10,599,127.82,2019-06-06,1 days,-0.115944
2019-06-06,MSFT2019-06-07P114.00,MSFT,2019-06-07,0.02,0.0,0.01,put,114.0,20,443,127.82,2019-06-06,1 days,-0.108121
2019-06-06,MSFT2019-06-07P115.00,MSFT,2019-06-07,0.02,0.0,0.01,put,115.0,46,1129,127.82,2019-06-06,1 days,-0.100297


In [6]:
# load LSTM and RandomForest df
lstm_signals_df = pd.read_pickle('Resources/msft_LSTM_signals_df.pickle')
rf_signals_df = pd.read_pickle('Resources/msft_RF_signals_df.pickle')

In [7]:
merged_lstm = df.merge(lstm_signals_df, left_index = True, right_index = True, how= 'inner')

In [8]:
merged = merged_lstm.merge(rf_signals_df, left_index =True, right_index =True, how = 'inner')

In [9]:
merged.head()

,OptionKey,Symbol,ExpirationDate,AskPrice,BidPrice,LastPrice,PutCall,StrikePrice,Volume,OpenInterest,UnderlyingPrice,DataDate,time delta,% out,LSTM Predicted Signal,RF Predicted Value
2019-11-13,MSFT2019-11-15P120.00,MSFT,2019-11-15,0.01,0.0,0.01,put,120.0,9,11238,147.31,2019-11-13,2 days,-0.185391,1,1
2019-11-13,MSFT2019-11-15P125.00,MSFT,2019-11-15,0.01,0.0,0.01,put,125.0,14,11164,147.31,2019-11-13,2 days,-0.151449,1,1
2019-11-13,MSFT2019-11-15P128.00,MSFT,2019-11-15,0.03,0.0,0.01,put,128.0,10,0,147.31,2019-11-13,2 days,-0.131084,1,1
2019-11-13,MSFT2019-11-15P130.00,MSFT,2019-11-15,0.01,0.0,0.02,put,130.0,490,20253,147.31,2019-11-13,2 days,-0.117507,1,1
2019-11-13,MSFT2019-11-15P132.00,MSFT,2019-11-15,0.04,0.0,0.01,put,132.0,5,1671,147.31,2019-11-13,2 days,-0.103930,1,1


In [10]:
#filter for OTM to make only entries to sell puts - not necessary

merged['LSTM OTM Cr Entry'] = np.where((merged['LSTM Predicted Signal'] ==1) &(merged['% out']> 0) & (merged['time delta'].map(lambda x:str(x).split(' ',1)[0]).astype(int) >= 7), merged.AskPrice*100, 0)
merged['RF OTM Cr Entry'] = np.where((merged['RF Predicted Value'] ==1) &(merged['% out']> 0) & (merged['time delta'].map(lambda x:str(x).split(' ',1)[0]).astype(int) >= 7), merged.AskPrice*100, 0)
merged['Bid Price'] = merged.BidPrice*100
merged = merged.sort_values(['OptionKey','DataDate'], ascending = True)

merged

,OptionKey,Symbol,ExpirationDate,AskPrice,BidPrice,LastPrice,PutCall,StrikePrice,Volume,OpenInterest,UnderlyingPrice,DataDate,time delta,% out,LSTM Predicted Signal,RF Predicted Value,LSTM OTM Cr Entry,RF OTM Cr Entry,Bid Price
2019-11-13,MSFT2019-11-15P120.00,MSFT,2019-11-15,0.01,0.00,0.01,put,120.0,9,11238,147.31,2019-11-13,2 days,-0.185391,1,1,0.0,0.0,0.0
2019-11-13,MSFT2019-11-15P125.00,MSFT,2019-11-15,0.01,0.00,0.01,put,125.0,14,11164,147.31,2019-11-13,2 days,-0.151449,1,1,0.0,0.0,0.0
2019-11-14,MSFT2019-11-15P125.00,MSFT,2019-11-15,0.01,0.00,0.01,put,125.0,43,11161,148.06,2019-11-14,1 days,-0.155748,1,1,0.0,0.0,0.0
2019-11-15,MSFT2019-11-15P125.00,MSFT,2019-11-15,0.01,0.00,0.01,put,125.0,29,11144,149.97,2019-11-15,0 days,-0.166500,1,1,0.0,0.0,0.0
2019-11-14,MSFT2019-11-15P127.00,MSFT,2019-11-15,0.01,0.00,0.01,put,127.0,2,7779,148.06,2019-11-14,1 days,-0.142240,1,1,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-02,MSFT2019-12-27P162.50,MSFT,2019-12-27,13.60,12.30,12.72,put,162.5,5,5,149.55,2019-12-02,25 days,0.086593,-1,1,0.0,1360.0,1230.0
2019-12-13,MSFT2019-12-27P162.50,MSFT,2019-12-27,10.15,6.45,9.05,put,162.5,8,0,154.53,2019-12-13,14 days,0.051576,1,1,1015.0,1015.0,645.0
2019-11-19,MSFT2019-12-27P170.00,MSFT,2019-12-27,21.70,18.15,19.60,put,170.0,1,0,150.39,2019-11-19,38 days,0.130394,-1,1,0.0,2170.0,1815.0
2019-11-20,MSFT2019-12-27P170.00,MSFT,2019-12-27,22.25,19.45,21.05,put,170.0,1,1,149.62,2019-11-20,37 days,0.136212,-1,1,0.0,2225.0,1945.0
